In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
import pandas as pd
import torch
from backtesting import Strategy

# Import your GRU model (assumes PyTorch Lightning or plain PyTorch model)
from models.gru_model import GRUModule
from models.transformer_model import TransformerModule

In [6]:
# Load the GRU model once globally (not in the strategy)
CHECKPOINT_PATH = '../lightning_logs/transformer_3barrier_clas/version_0/checkpoints/best_checkpoint.ckpt'
model = TransformerModule.load_from_checkpoint(CHECKPOINT_PATH)
model.eval()
model.to('cpu')  # or 'cuda' if you're running with GPU

TransformerModule(
  (model): TransformerModel(
    (input_proj): Linear(in_features=6, out_features=128, bias=True)
    (positional_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=256, bias=True)
          (dropout): Dropout(p=0.3, inplace=False)
          (linear2): Linear(in_features=256, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.3, inplace=False)
          (dropout2): Dropout(p=0.3, inplace=False)
        )
      )
    )
    (fc_out): Linear(in_features=1

In [7]:
class GRUStrategy(Strategy):
    def init(self):
        self.sequence_length = 30
        self.lot_size = 0.1
        self.stop_loss_amount = 0.05
        self.take_profit_amount = 0.075

        self.buy_count = 0
        self.sell_count = 0

    def next(self):
        if len(self.data.Close) < self.sequence_length + 1:
            return  # Not enough data to predict

        # Extract the last N closes
        close_prices = self.data.Close[-self.sequence_length:]
        close_returns = pd.Series(close_prices).pct_change().dropna().values.reshape(1, -1, 1)
        close_tensor = torch.FloatTensor(close_returns)

        # Model prediction
        with torch.no_grad():
            _, probs = model(close_tensor)
            signal = torch.argmax(probs, dim=1).item()
            if probs[0, signal] < 0.7:
                return
        current_price = self.data.Close[-1]

        # 0 = sell, 1 = hold, 2 = buy
        if signal == 2:
            sl = current_price - self.stop_loss_amount
            tp = current_price + self.take_profit_amount
            self.buy(size=self.lot_size, sl=sl, tp=tp)
            self.buy_count += 1

        if signal == 0:
            sl = current_price + self.stop_loss_amount
            tp = current_price - self.take_profit_amount
            self.sell(size=self.lot_size, sl=sl, tp=tp)
            self.sell_count += 1

In [8]:
import torch
from backtesting import Strategy

class LongOnlyStrategy(Strategy):
    def init(self):
        self.sequence_length = 30
        self.lot_size = 0.1
        self.stop_loss_amount = 0.005
        self.take_profit_amount = 0.005
        self.conf_threshold = 0

        self.buy_count = 0
        self.sell_count = 0

        # Setup model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device).eval()

    def next(self):
        if len(self.data) < self.sequence_length + 1:
            return  # Not enough data for prediction

        # Extract the latest window of feature data
        df_window = self.data.df.iloc[-self.sequence_length:]

        try:
            X = df_window[[
                'close_log_return_scaled',
                'log_volume_scaled',
                'ret_mean_5',
                'ret_mean_10',
                'rsi',
                'macd_diff'
            ]].values.astype('float32')  # shape: (30, 6)
        except KeyError as e:
            print(f"Missing expected feature: {e}")
            return

        # Prepare tensor
        X_tensor = torch.from_numpy(X).unsqueeze(0).to(self.device)  # shape: (1, seq_len, n_features)

        with torch.no_grad():
            _, logits = self.model(X_tensor)
            probs = torch.softmax(logits, dim=1)
            signal = torch.argmax(probs, dim=1).item()
            confidence = probs[0, signal].item()

        # Only act on confident BUY signal
        if signal == 2 and confidence >= self.conf_threshold:
            current_price = self.data.Close[-1]
            sl = current_price - self.stop_loss_amount
            tp = current_price + self.take_profit_amount

            self.buy(size=self.lot_size, sl=sl, tp=tp)
            self.buy_count += 1


In [9]:
PKL_PATH = '../data/processed/usdjpy-h1-bar-2019-01-01-2025-05-12_processed.pkl'

df = pd.read_pickle(PKL_PATH)
df = df[df['timestamp'].dt.year <= 2024].copy()
df.head()

,timestamp,open,high,low,close,volume,time,log_volume,log_volume_scaled,time_group,...,ma_21,ret_mean_5,ret_mean_10,label,train_label,macd,macd_signal,macd_diff,rsi,log_dist_from_ma
102,2019-01-08 04:00:00,108.878,108.997,108.876,108.987,7157.2202,2019-01-08 04:00:00,8.876017,-0.099633,2,...,108.543476,0.000664,0.000385,-1,0,0.149854,0.119898,0.029956,76.065066,0.004078
103,2019-01-08 05:00:00,108.987,108.996,108.780,108.785,7517.8198,2019-01-08 05:00:00,8.925164,-0.049506,2,...,108.571571,0.000092,0.000174,-1,0,0.146038,0.125126,0.020912,62.755102,0.001964
104,2019-01-08 06:00:00,108.784,108.889,108.736,108.883,5566.1499,2019-01-08 06:00:00,8.624639,-0.356019,2,...,108.604286,0.000594,0.000163,-1,0,0.149201,0.129941,0.019260,60.171473,0.002563
105,2019-01-08 07:00:00,108.882,109.087,108.826,108.981,12419.2402,2019-01-08 07:00:00,9.427083,0.462412,2,...,108.634333,0.000494,0.000254,-1,0,0.157797,0.135512,0.022285,63.829787,0.003186
106,2019-01-08 08:00:00,108.979,109.018,108.880,108.911,10538.8896,2019-01-08 08:00:00,9.262922,0.294981,2,...,108.660762,0.000061,0.000236,-1,0,0.157149,0.139840,0.017310,62.276306,0.002300


In [10]:
from backtesting import Backtest

In [11]:
# Format the DataFrame for backtesting.py
df['time'] = pd.to_datetime(df['timestamp'])

df.set_index('time', inplace=True)
df.rename(columns={
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'volume': 'Volume'
}, inplace=True)

print(df.head())

# Run backtest
test = Backtest(df, LongOnlyStrategy, cash=100000, hedging=True, exclusive_orders=True)
result = test.run()

print(result)
print(f'Buy count = {result._strategy.buy_count}')
print(f'Sell count = {result._strategy.sell_count}')

                              timestamp     Open     High      Low    Close  \
time                                                                          
2019-01-08 04:00:00 2019-01-08 04:00:00  108.878  108.997  108.876  108.987   
2019-01-08 05:00:00 2019-01-08 05:00:00  108.987  108.996  108.780  108.785   
2019-01-08 06:00:00 2019-01-08 06:00:00  108.784  108.889  108.736  108.883   
2019-01-08 07:00:00 2019-01-08 07:00:00  108.882  109.087  108.826  108.981   
2019-01-08 08:00:00 2019-01-08 08:00:00  108.979  109.018  108.880  108.911   

                         Volume  log_volume  log_volume_scaled  time_group  \
time                                                                         
2019-01-08 04:00:00   7157.2202    8.876017          -0.099633           2   
2019-01-08 05:00:00   7517.8198    8.925164          -0.049506           2   
2019-01-08 06:00:00   5566.1499    8.624639          -0.356019           2   
2019-01-08 07:00:00  12419.2402    9.427083           0.

Backtest.run:   0%|          | 0/27449 [00:00<?, ?bar/s]

Start                     2019-01-08 04:00:00
End                       2024-12-20 21:00:00
Duration                   2173 days 17:00:00
Exposure Time [%]                    49.93443
Equity Final [$]                    95411.655
Equity Peak [$]                      100000.0
Return [%]                           -4.58834
Buy & Hold Return [%]                43.50978
Return (Ann.) [%]                     -0.9655
Volatility (Ann.) [%]                 0.05546
CAGR [%]                             -0.54304
Sharpe Ratio                        -17.40832
Sortino Ratio                       -11.64485
Calmar Ratio                         -0.21042
Alpha [%]                             -4.5933
Beta                                  0.00011
Max. Drawdown [%]                    -4.58834
Avg. Drawdown [%]                    -4.58834
Max. Drawdown Duration     2172 days 07:00:00
Avg. Drawdown Duration     2172 days 07:00:00
# Trades                                13707
Win Rate [%]                      

In [ ]:
test.plot()

In [ ]:
test.plot(
    results=None,
    filename=None,
    plot_width=None,
    plot_equity=True,
    plot_return=False,
    plot_pl=False,
    plot_volume=False,
    plot_drawdown=False,
    plot_trades=False,
    smooth_equity=False,
    relative_equity=True,
    superimpose=False,
    resample=True,
    reverse_indicators=False,
    show_legend=False,
    open_browser=False
)